# State Farm Distracted Driver Detection: Take 3
_Can computer vision spot distracted drivers?_

---

## Lesson 3 Homework Assignment

Dataset: https://www.kaggle.com/c/state-farm-distracted-driver-detection

### Dealing with Overfitting
In [`lesson2-hmwk.ipynb`](https://github.com/iconix/fast.ai/blob/master/nbs/lesson2-hmwk.ipynb), my final results (after 5 epochs) were as follows:
> `loss: 0.6260 - acc: 0.7907 - val_loss: 1.6719 - val_acc: 0.4978`

When `val_acc >> acc`, this is a clear indicator of overfitting on the training data.

On the bright side, this means that my neural net architecture is complex enough to model the data. The next step is to generalize my architecture a bit more.

Here is the prioritized list of approaches to reducing overfitting provided during class:
1. Add more data
2. Use data augmentation
3. Use architectures that generalize well
4. Add regularization (dropout, L2/L1 regularization)
5. Reduce architecture complexity

### Downloading and creating the datasets

This week, I am skipping the download, split, and create of training, validation, test, and sample datasets, relying instead on the data split from last week. See [`lesson2-hmwk.ipynb`](https://github.com/iconix/fast.ai/blob/master/nbs/lesson2-hmwk.ipynb) for those steps, if needed.

Additionally, I will be starting with the weights from last week below (`finetune2.h5`).

### Basic Configuration

In [2]:
import os

current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir + '/data/statefarm/'

# point to your training images
train_dir = DATA_HOME_DIR + 'train'

# point to the 'driver_imgs_list.csv'
lookup = DATA_HOME_DIR + 'driver_imgs_list.csv'

# point to the validation directory, which will be created in the next block
val_dir = DATA_HOME_DIR + 'valid'

sample_dir = DATA_HOME_DIR + 'sample'

test_dir = DATA_HOME_DIR + 'test'

#path = DATA_HOME_DIR + 'sample/'
path = DATA_HOME_DIR
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [3]:
%matplotlib inline

In [4]:
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [5]:
batch_size=64

### Load pre-trained model layers

In [8]:
def get_week2_model():
    model = vgg_ft(10)
    finetune2_path = model_path+'finetune2.h5'
    model.load_weights(finetune2_path);
    return model

In [11]:
model = get_week2_model()

In [12]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_3 (Lambda)                (None, 3, 224, 224)   0           lambda_input_3[0][0]             
____________________________________________________________________________________________________
zeropadding2d_27 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_3[0][0]                   
____________________________________________________________________________________________________
convolution2d_27 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_27[0][0]           
____________________________________________________________________________________________________
zeropadding2d_28 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_27[0][0]           
___________________________________________________________________________________________

In [13]:
layers = model.layers
last_conv_idx = [index for index,layer in enumerate(layers) 
                    if type(layer) is Convolution2D][-1]

conv_layers = layers[:last_conv_idx+1]
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

### Data Augmentation

Define a generator that includes data augmentation (convenient feature of Keras).

In [30]:
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, 
       height_shift_range=0.1, shear_range=0.15, zoom_range=0.1, 
       channel_shift_range=10., horizontal_flip=True)

In [31]:
trn_batches = get_batches(path+'train', gen, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 18298 images belonging to 10 classes.
Found 4126 images belonging to 10 classes.


Jeremy's [explanation](http://forums.fast.ai/t/lesson-3-discussion/186/33) as to why we aren't training the convolutional layers here:
> The early layers are so general (e.g. remember Zeiler's visualizations - layer 1 just finds edges and gradients) that it's extremely unlikely that you'll need to change them, unless you're looking at very different kinds of images. e.g. if you're classifying line drawings, instead of photos, you'll probably need to retrain many conv layers too.

In [ ]:
for layer in layers[:last_conv_idx+1]: layer.trainable=False

In [ ]:
# Updating slowly because it is finely tuned
K.set_value(model.optimizer.lr, 0.00001)

In [ ]:
model.fit_generator(trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=8, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

After 8 epochs, we are overfitting much much less, which is great!
> `loss: 1.9278 - acc: 0.5002 - val_loss: 1.7463 - val_acc: 0.5099`

In fact, now we're underfitting very slightly. This seems like a good time to try decreasing dropout a smidge, just to see what happens.

Let's save our intermediate weights first, just in case.

In [6]:
finetune_hw3_1_path = model_path+'finetune_hw3_1.h5'
if not os.path.exists(finetune_hw3_1_path):
    model.save_weights(finetune_hw3_1_path)
model.load_weights(finetune_hw3_1_path)

#### Intermediate Kaggle submission

Interestingly, despite the significant decrease in overfitting, `val_acc` only improved a small amount from `0.4978`... I'd like to try submitting these results to Kaggle, just to see how this result reflects in the rankings.

In [22]:
from IPython.display import FileLink

subm_name = 'subm_hmwk3_1.gz'
subm_path = path + 'results/' + subm_name

#Get the classes from the validation batch
val_preprocess = get_batches(path+'valid', shuffle=False, batch_size=1)
classes = sorted(val_preprocess.class_indices, key=val_preprocess.class_indices.get)

Found 4126 images belonging to 10 classes.


Note on `.predict` vs `.predict_generator`: "the precomputed data will be large. Then it is likely that you will encounter errors such as OOM or **kernel death** during training. In this case, you might want to use model.fit_generator() instead of model.fit()" ([source](http://forums.fast.ai/t/fine-tuning-vgg-taking-very-long/3825/11)) - I kept running into kernel death with `.predict`...

In [23]:
test_batches = get_batches(path+'test', shuffle=False, batch_size = batch_size)
preds = model.predict_generator(test_batches, test_batches.nb_sample)

Found 79726 images belonging to 1 classes.


In [24]:
# (number of classes - 1) = 9 (http://forums.fast.ai/t/moving-up-the-ncfm-leaderboard-by-100-positions-do-clip/1035/6)
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [51]:
subm = do_clip(preds, 0.9) # still unsure how to set mx

In [52]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.011417,0.011111,0.011111,0.011111,0.011111,0.463585,0.020937,0.011111,0.045846,0.443723
1,img_14887.jpg,0.346917,0.121693,0.011111,0.098112,0.011111,0.061683,0.047707,0.017239,0.234232,0.060724
2,img_62885.jpg,0.011111,0.011111,0.423249,0.015064,0.147754,0.044589,0.280264,0.011111,0.083560,0.011111
3,img_45125.jpg,0.011111,0.011111,0.256628,0.011111,0.011111,0.011111,0.133316,0.011111,0.593049,0.011111
4,img_22633.jpg,0.031959,0.011320,0.084662,0.011111,0.011111,0.034428,0.477008,0.011111,0.323893,0.026945


In [53]:
submission.to_csv(subm_path, index=False, compression='gzip')
FileLink(subm_path)

/home/ubuntu/hmwk/nbs/data/statefarm/results/subm_hmwk3_1.gz

Private Score: `1.41527` ... comparable to my week 2 results.

And interestingly, weeks ago, my Public Score tended to be the lower/better score, or at least comparable; now, it is `1.48239` in comparison. See [this Quora post](https://www.quora.com/What-is-the-difference-between-public-and-private-leaderboard-in-Kaggle/answer/Giuliano-Janson) for an explanation:
> The public LB is computed on a portion of the test set, the private is computed on the remainder of the test set (not the whole test set).
"Fitting the LB" is a Kaggle term used to describe when you're tuning your models to perform well on the public LB. There is an art and a science in doing so and experience Kagglers are able to make the most out of it without overfitting. If not done well, that usually lends itself to worse scores on the private LB, sometimes disasters. In general the key is to build a model that generalizes well.

### Data Augmentation + Decrease Dropout
We're going to experiment with decreasing the dropout rate from Vgg16's 50% to 25%.

In [36]:
# will start again with conv_layers from last week's model
model = get_week2_model()

In [42]:
def get_fc_model(fcl, conv_input_shape):
    m = Sequential([
        MaxPooling2D(input_shape=conv_input_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.25),
        Dense(4096, activation='relu'),
        Dropout(0.25),
        Dense(10, activation='softmax')
        ])

    for l1,l2 in zip(m.layers, fcl): l1.set_weights(l2.get_weights())

    m.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return m

In [43]:
layers = model.layers
last_conv_idx = [index for index,layer in enumerate(layers) 
                    if type(layer) is Convolution2D][-1]

conv_layers = layers[:last_conv_idx+1]
fc_layers = layers[last_conv_idx+1:]

In [44]:
conv_model = Sequential(conv_layers)
fc_model = get_fc_model(fc_layers, conv_layers[-1].output_shape[1:])

Add the new `fc_model` with less dropout to the original `conv_model` from last week...

In [45]:
for layer in conv_model.layers: layer.trainable=False
conv_model.add(fc_model)

Then compile the new combined model and fit!

In [48]:
conv_model.compile(optimizer=Adam(0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [49]:
conv_model.fit_generator(trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=8, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/8
18298/18298 [==============================] - 549s - loss: 0.5896 - acc: 0.8057 - val_loss: 1.5074 - val_acc: 0.5880
Epoch 2/8
18298/18298 [==============================] - 549s - loss: 0.2387 - acc: 0.9200 - val_loss: 1.3920 - val_acc: 0.6156
Epoch 3/8
18298/18298 [==============================] - 549s - loss: 0.1575 - acc: 0.9501 - val_loss: 1.2786 - val_acc: 0.6512
Epoch 4/8
18298/18298 [==============================] - 550s - loss: 0.1248 - acc: 0.9601 - val_loss: 1.4503 - val_acc: 0.6006
Epoch 5/8
18298/18298 [==============================] - 549s - loss: 0.0931 - acc: 0.9692 - val_loss: 1.5553 - val_acc: 0.6156
Epoch 6/8
18298/18298 [==============================] - 550s - loss: 0.0815 - acc: 0.9750 - val_loss: 1.2619 - val_acc: 0.6854
Epoch 7/8
18298/18298 [==============================] - 550s - loss: 0.0675 - acc: 0.9787 - val_loss: 1.4714 - val_acc: 0.6432
Epoch 8/8
18298/18298 [==============================] - 550s - loss: 0.0599 - acc: 0.9797 - val_loss: 1

In [52]:
conv_model.optimizer.lr = 0.001 # back to default learning rate
conv_model.fit_generator(trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/1
18298/18298 [==============================] - 550s - loss: 0.0554 - acc: 0.9819 - val_loss: 1.3206 - val_acc: 0.7038


Drumroll...

    `loss: 0.0554 - acc: 0.9819 - val_loss: 1.3206 - val_acc: 0.7038`

Wow, back to massive overfitting. But I was expecting this since I reduced dropout in a model that was already pretty well balanced in fit thanks to data augmentation in the last section.

Fortunately as I was also hoping, there has been a _huge_ increase in the upper bounds of both `acc` and `val_acc` - dropout was stunting the model's learning ability! So I'm hoping this means that my next final attempt at reducing overfitting will work again, but with much higher accuracies than the `~0.5` I was seeing earlier.

Let's save weights again.

In [54]:
finetune_hw3_2_path = model_path+'finetune_hw3_2.h5'
if not os.path.exists(finetune_hw3_2_path):
    conv_model.save_weights(finetune_hw3_2_path)

### Add Batch Normalization

In [ ]:
# start with the model just trained above
model = vgg_ft(10)
model.load_weights(finetune_hw3_2_path)

I'm now going to assume that the convolution layers are well-trained enough that I can pre-calculate them. This means that Data Augmentation will not affect the conv layers.

In [ ]:
val_classes = val_batches.classes
trn_classes = trn_batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [ ]:
layers = model.layers
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)

In [ ]:
trn_features = conv_model.predict_generator(batches, batches.nb_sample)
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [ ]:
train_convlayer_feats_path = model_path+'hmwk3_train_convlayer_features.bc'
if not os.path.exists(train_convlayer_feats_path):
    save_array(train_convlayer_feats_path, trn_features)
    
valid_convlayer_feats_path = model_path+'hmwk3_valid_convlayer_features.bc'
if not os.path.exists(valid_convlayer_feats_path):
    save_array(valid_convlayer_feats_path, val_features)

In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(1000, activation='softmax')
        ]

In [ ]:
def load_fc_weights_from_vgg16bn(m):
    "Load weights for model from the dense layers of the Vgg16BN model."
    # See imagenet_batchnorm.ipynb for info on how the weights for
    # Vgg16BN can be generated from the standard Vgg16 weights.
    from vgg16bn import Vgg16BN
    vgg16_bn = Vgg16BN()
    _, fcl = split_at(vgg16_bn.model, Convolution2D)
    copy_weights(fcl, m.layers)

In [ ]:
bn_fc_model = Sequential(get_bn_layers(0.25))

In [ ]:
load_fc_weights_from_vgg16bn(bn_fc_model)

In [ ]:
bn_fc_model.pop()
for layer in bn_fc_model.layers: layer.trainable=False

In [ ]:
bn_fc_model.add(Dense(10,activation='softmax'))

In [ ]:
bn_fc_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_fc_model.fit(trn_features, trn_labels, nb_epoch=8, validation_data=(val_features, val_labels))

In [ ]:
finetune_hw3_3_path = model_path+'finetune_hw3_3.h5'
if not os.path.exists(finetune_hw3_3_path):
    bc_fc_model.save_weights(finetune_hw3_3_path)

In [ ]:
final_model = Sequential(conv_layers)
for layer in final_model.layers: layer.trainable = False
final_model.add(bn_fc_model)